In [ ]:
import json
import os
import uuid
import cv2
from matplotlib import pyplot as plt
import matplotlib.patches as patches

from thumbnail_selector import PEN_SITE_MAPPING, get_capture_keys, get_image_urls_and_crop_metadatas, \
    get_random_image_urls_and_crop_metadatas
from construct_fish_detection_dataset_o2kr2 import establish_plali_connection, insert_into_plali


from video_service.video_generation import generate_video
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils

<h1> Set up stage 1 annotation </h1>

In stage 1, we will only identify which 512x512 left images contain a full fish. These will then cascade onto stage 2 for full key-point annotation 

<h1> Prepare image URL data for 2020-02-13 </h1>

In [ ]:
generate_video(89, '2020-02-13', 0, 24, 1.0, True)

In [ ]:
INBOUND_BUCKET = 'aquabyte-images-raw'

def get_image_urls(capture_keys):
    """Gets left urls, right urls, and crop metadatas corresponding to capture keys."""

    left_urls, crop_metadatas = [], []
    for capture_key in capture_keys:

        # get image URLs
        left_image_key = capture_key.replace('capture.json', 'left_frame.resize_512_512.jpg')
        left_image_url = os.path.join('s3://', INBOUND_BUCKET, left_image_key)
        left_urls.append(left_image_url)

    return left_urls


In [ ]:
pen_id, start_date, end_date = 89, '2020-02-13', '2020-02-13'
capture_keys = get_capture_keys(pen_id, start_date, end_date, inbound_bucket='aquabyte-images-raw')

In [ ]:
hours_to_include = [7, 8, 9, 10, 12, 13, 14, 15, 16]
capture_keys_to_include = []
for capture_key in capture_keys:
    key_components = capture_key.split('/')
    hour_component = [component for component in key_components if 'hour=' in component][0]
    hour = int(hour_component[-2:])
    if hour in hours_to_include:
        capture_keys_to_include.append(capture_key)
        
image_urls = get_image_urls(capture_keys_to_include)


<h2> Insert into PLALI </h1>

In [ ]:
def process_into_plali_records(image_urls):

    values_to_insert = []
    for idx, image_url in enumerate(image_urls):
        id = str(uuid.uuid4())
        images = {image_url}
        metadata = {}
        priority = float(idx) / len(image_urls)

        values = {
            'id': id,
            'workflow_id': '00000000-0000-0000-0000-000000000047',
            'images': images,
            'metadata': metadata,
            'priority': priority
        }

        values_to_insert.append(values)

    return values_to_insert

In [ ]:
values_to_insert = process_into_plali_records(image_urls)

In [ ]:
os.environ['PLALI_SQL_CREDENTIALS'] = '/run/secrets/plali_sql_credentials.json'
engine, sql_metadata = establish_plali_connection()

In [ ]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [ ]:
n = 10
count = 0
for chunk in chunker(values_to_insert[2:], n):
    insert_into_plali(chunk, engine, sql_metadata)
    
    count += 1
    print(count)

<h1> Prepare image URL data for 2020-02-14 </h1>

In [ ]:
generate_video(89, '2020-02-14', 0, 24, 1.0, True)

In [ ]:
pen_id, start_date, end_date = 89, '2020-02-14', '2020-02-14'
capture_keys = get_capture_keys(pen_id, start_date, end_date, inbound_bucket='aquabyte-images-raw')

In [ ]:
hours_to_include = [6, 7, 8, 9, 10, 11, 12]
capture_keys_to_include = []
for capture_key in capture_keys:
    key_components = capture_key.split('/')
    hour_component = [component for component in key_components if 'hour=' in component][0]
    hour = int(hour_component[-2:])
    
    if hour == 6:
        ts_component = [component for component in key_components if 'at=' in component][0]
        minute = int(ts_component.split(':')[1])
        if minute < 35: 
            continue
    
    if hour == 10:
        ts_component = [component for component in key_components if 'at=' in component][0]
        minute = int(ts_component.split(':')[1])
        if minute > 30: 
            continue
    if hour == 11:
        ts_component = [component for component in key_components if 'at=' in component][0]
        minute = int(ts_component.split(':')[1])
        if minute < 28: 
            continue
    
    if hour in hours_to_include:
        capture_keys_to_include.append(capture_key)
        
image_urls = get_image_urls(capture_keys_to_include)


In [ ]:
values_to_insert = process_into_plali_records(image_urls)

In [ ]:
n = 10
count = 0
for chunk in chunker(values_to_insert[2:], n):
    insert_into_plali(chunk, engine, sql_metadata)
    
    count += 1
    print(count)

<h1> Complete right image annotations corresponding to left images that had a full fish bounding box <h1>

In [ ]:
rds = RDSAccessUtils(json.load(open(os.environ['PLALI_SQL_CREDENTIALS'])))
query = """
    select * from plali.plali_annotations x
    inner join 
    ( select a.id as plali_image_id, a.images, a.metadata, b.id as workflow_id, b.name from plali.plali_images a
    inner join plali.plali_workflows b
    on a.workflow_id = b.id ) y
    on x.plali_image_id = y.plali_image_id
    where workflow_id = '00000000-0000-0000-0000-000000000047';
"""

annotated_df = rds.extract_from_database(query)

<h2> Confirm from random subset that annotations are well-behaved </h2>

In [ ]:
s3 = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))

In [ ]:
annotated_df['annotation_count'] = annotated_df.annotation.apply(lambda x: len(x['annotations']) if x.get('annotations') else 0)

In [ ]:
# filter only on images in which full fish was found
mask = annotated_df.annotation_count == 1
for idx, row in annotated_df[mask].head(10).iterrows():
    
    # download image
    image_s3_url = row.images[0]
    url_components = image_s3_url.replace('s3://', '').split('/')
    bucket = url_components[0]
    key = os.path.join(*url_components[1:])
    image_f = s3.download_from_s3(bucket, key)
    
    # plot image
    fig, ax = plt.subplots()
    im = cv2.imread(image_f)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    ax.imshow(im)
    
    # plot rectangle
    ann = row.annotation['annotations'][0]
    x, y, w, h = ann['xCrop'], ann['yCrop'], ann['width'], ann['height']
    rect = patches.Rectangle((x, y), w, h, linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
    
    plt.show()
    
    

    
    

<h2> Get list of right image URLs for full fish </h2>

In [ ]:
image_s3_url

In [ ]:
right_image_s3_urls = []

mask = annotated_df.annotation_count == 1
for idx, row in annotated_df[mask].iterrows():
    image_s3_url = row.images[0]
    if 'left_frame' in image_s3_url:
        right_image_s3_url = image_s3_url.replace('left_frame', 'right_frame')
        right_image_s3_urls.append(right_image_s3_url)
    


In [ ]:
values_to_insert = process_into_plali_records(right_image_s3_urls)

In [ ]:
n = 10
count = 0
for chunk in chunker(values_to_insert, n):
    insert_into_plali(chunk, engine, sql_metadata)
    
    count += 1
    print(count)

In [ ]:
list(values_to_insert[0]['images'])[0]

In [ ]:
n = 10
count = 0
for chunk in chunker(values_to_insert, n):
    insert_into_plali(chunk, engine, sql_metadata)
    
    count += 1
    print(count)

In [ ]:
values_to_insert_modified = []
for v in values_to_insert:
    v_modified = dict(v)
    v_modified['images'] = list(v['images'])
    values_to_insert_modified.append(v_modified)

In [ ]:
json.dump(values_to_insert_modified, open('/root/data/alok/biomass_estimation/playground/values_to_insert.json', 'w'))